# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,howrah,22.5892,88.3103,39.07,29,2,3.63,IN,1714189405
1,1,maine soroa,13.2114,12.0241,29.41,15,99,2.24,NE,1714189406
2,2,lompoc,34.6391,-120.4579,12.64,87,46,8.20,US,1714189318
3,3,iquique,-20.2208,-70.1431,18.33,83,59,0.45,CL,1714189407
4,4,blackmans bay,-43.0167,147.3167,14.89,61,53,1.43,AU,1714189251


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
)


# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness
# Drop any rows with null values
ideal_cities_df=city_data_df.loc[(city_data_df['Max Temp'] <27) & 
    (city_data_df['Max Temp'] >21) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)].dropna()
# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
73,73,kingston,17.9970,-76.7936,24.00,80,0,1.55,JM,1714189454
187,187,namibe,-15.1961,12.1522,24.24,76,0,0.92,AO,1714189534
204,204,remire-montjoly,4.9167,-52.2667,26.02,89,0,1.54,GF,1714189304
344,344,turaif,31.6725,38.6637,21.14,17,0,4.12,SA,1714189344
428,428,cruzeiro do sul,-29.5128,-51.9853,23.62,93,0,1.87,BR,1714189693
446,446,tonghae,37.5439,129.1069,22.85,49,0,2.26,KR,1714189695
462,462,bandar abbas,27.1865,56.2808,25.99,53,0,3.09,IR,1714189421


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
73,kingston,JM,17.9970,-76.7936,80,
187,namibe,AO,-15.1961,12.1522,76,
204,remire-montjoly,GF,4.9167,-52.2667,89,
344,turaif,SA,31.6725,38.6637,17,
428,cruzeiro do sul,BR,-29.5128,-51.9853,93,
446,tonghae,KR,37.5439,129.1069,49,
462,bandar abbas,IR,27.1865,56.2808,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "limit":20,
    "categories": "accommodation" 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kingston - nearest hotel: Aphrodite Hotel
namibe - nearest hotel: Pensão Delta do Deserto
remire-montjoly - nearest hotel: Motel du Lac
turaif - nearest hotel: المودة
cruzeiro do sul - nearest hotel: Estrela Palace Hotel
tonghae - nearest hotel: 부성모텔
bandar abbas - nearest hotel: ساختمان همت


,City,Country,Lat,Lng,Humidity,Hotel Name
73,kingston,JM,17.9970,-76.7936,80,Aphrodite Hotel
187,namibe,AO,-15.1961,12.1522,76,Pensão Delta do Deserto
204,remire-montjoly,GF,4.9167,-52.2667,89,Motel du Lac
344,turaif,SA,31.6725,38.6637,17,المودة
428,cruzeiro do sul,BR,-29.5128,-51.9853,93,Estrela Palace Hotel
446,tonghae,KR,37.5439,129.1069,49,부성모텔
462,bandar abbas,IR,27.1865,56.2808,53,ساختمان همت


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
city_hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
city_hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)